# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [35]:
!cat publications.tsv





2022-03-18	Reconstructing codependent cellular cross-talk in lung adenocarcinoma using REMI	Science Advances		"Alice Yu, Yuanyuan Li, Irene Li, Michael G Ozawa, Christine Y Yeh, Aaron E Chiou, Winston L Trope, Jonathan Taylor, Joseph Shrager, Sylvia K Plevritis"	2022_Science_Advances	http://christineyyeh.github.io/files/2022_Science_Advances.pdf

## Import pandas

We are using the very handy pandas library for dataframes.

In [36]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [66]:
publications = pd.read_csv("../markdown_generator/publications.csv")
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url,doi
0,2018-02-28,Integrative personal omics profiles during per...,Cell Systems,"""""","Brian D Piening, Wenyu Zhou, Kévin Contrepois,...",2018_Cell_Systems,http://christineyyeh.github.io/files/2018_Cell...,https://doi.org/10.1016/j.cels.2017.12.013
1,2017-11-07,Vitamin D supplementation decreases serum 27-h...,Breast Cancer Research and Treatment,"""""","Catherine C Going, Ludmila Alexandrova, Kennet...",2017_BCRT,http://christineyyeh.github.io/files/2017_BCRT...,https://doi.org/10.1007/s10549-017-4562-4
2,2017-10-17,Assessing biological and technological variabi...,Biomarker Research,"""""","<u>**Christine Y Yeh**<u>, Ravali Adusumilli, ...",2017_Biomarker_Research,http://christineyyeh.github.io/files/2017_Biom...,https://doi.org/10.1186/s40364-017-0110-y
3,2022-03-18,Reconstructing codependent cellular cross-talk...,Science Advances,"""""","Alice Yu, Yuanyuan Li, Irene Li, Michael G Oza...",2022_Science_Advances,http://christineyyeh.github.io/files/2022_Scie...,https://www.science.org/doi/10.1126/sciadv.abi...


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [67]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [68]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.doi + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
        
    md += "\n" + item.citation
    
    if len(str(item.paper_url)) > 5:
        md += "\n\n[Download PDF Here](" + item.paper_url + ")\n" 
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [69]:
!ls ../_publications/

2017-10-17-2017_Biomarker_Research.md 2018-02-28-2018_Cell_Systems.md
2017-11-07-2017_BCRT.md               2022-03-18-2022_Science_Advances.md


In [70]:
!cat ../_publications/2022-03-18-2022_Science_Advances.md

---
title: "Reconstructing codependent cellular cross-talk in lung adenocarcinoma using REMI"
collection: publications
permalink: /publication/2022-03-18-2022_Science_Advances
date: 2022-03-18
venue: 'Science Advances'
paperurl: 'https://www.science.org/doi/10.1126/sciadv.abi4757'
---
Alice Yu, Yuanyuan Li, Irene Li, Michael G Ozawa, <u>**Christine Y Yeh**<u>, Aaron E Chiou, Winston L Trope, Jonathan Taylor, Joseph Shrager, Sylvia K Plevritis

[Download PDF Here](http://christineyyeh.github.io/files/2022_Science_Advances.pdf)
